In [5]:
from pathlib import Path

# 1️⃣ 기본 경로 설정
AI_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/0_RAW_DATA/AI_dataset")
OUTPUT_TXT = AI_DIR / "titles.txt"

# 2️⃣ 모든 파일 재귀 탐색 후 확장자 필터링 (.mp4 / .mov 대소문자 무시)
video_exts = [".mp4", ".mov"]
all_files = [f for f in AI_DIR.rglob("*") if f.suffix.lower() in video_exts]

# 3️⃣ Synology 캐시(@eaDir, @SynoEAStream) 제외
valid_files = [
    f for f in all_files
    if "@eaDir" not in f.parts and not f.name.endswith("@SynoEAStream")
]

print(f"📂 Found {len(valid_files)} valid video files")

# 4️⃣ 파일 이름(확장자 제외)만 추출
titles = {f.stem for f in valid_files if f.is_file()}

# 5️⃣ 중복 제거 후 정렬
titles_sorted = sorted(titles)

# 6️⃣ txt 파일로 저장
with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
    for t in titles_sorted:
        f.write(t + "\n")

print(f"✅ Saved {len(titles_sorted)} unique titles to: {OUTPUT_TXT}")


📂 Found 944 valid video files
✅ Saved 110 unique titles to: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/0_RAW_DATA/AI_dataset/titles.txt


In [6]:
from pathlib import Path
import re
import pandas as pd

# 1️⃣ titles.txt 불러오기
titles_path = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/0_RAW_DATA/AI_dataset/titles.txt")  # titles.txt 경로
titles = [line.strip() for line in open(titles_path, encoding="utf-8").read().splitlines() if line.strip()]

# 2️⃣ 숫자패턴 추출용 정규식 (1,2,3을 뒷부분에서 탐색)
num_pattern = re.compile(r"(\d+)$")

# 3️⃣ 오탈자 및 불규칙 탐지
mapping_suggestions = {}
for t in titles:
    lower = t.lower()  # 소문자 변환
    fixed = re.sub(r"[\s]+", "_", lower)  # 띄어쓰기 → _
    fixed = re.sub(r"__+", "_", fixed)  # 중복 _
    fixed = fixed.replace("bobathtabale", "bobath_table")  # 명백한 오타
    fixed = fixed.replace("bobathtable", "bobath_table")   # 통일
    fixed = fixed.replace("pecdecfly", "pec_dec_fly")      # 불규칙 통일
    fixed = fixed.replace("brige", "bridge")               # 오타 수정
    fixed = fixed.replace("curl_", "curl")                 # 불필요한 _
    fixed = fixed.replace("q-setting", "q_setting")        # 하이픈 통일

    # 숫자 패턴 추출 및 접미사(1→frontal, 2→lateral, 3→diagonal) 변환
    match = num_pattern.search(fixed)
    if match:
        num = match.group(1)
        name = fixed[: -len(num)]
        angle = {"1": "frontal", "2": "lateral", "3": "diagonal"}.get(num, "")
        fixed_name = f"{angle}__{name.strip('_')}.mp4"
    else:
        fixed_name = f"frontal__{fixed}.mp4"  # 기본 frontal로 가정

    if fixed_name != t:
        mapping_suggestions[t] = fixed_name

# 4️⃣ 매핑 결과 DataFrame 생성
df_map = pd.DataFrame(mapping_suggestions.items(), columns=["original", "suggested"])
print(f"총 {len(df_map)}개 항목 제안됨")
print(df_map.head(20))

# 5️⃣ 검수용 CSV로 저장
output_csv = titles_path.parent / "mapping_suggestions.csv"
df_map.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ 매핑 제안 CSV 저장 완료: {output_csv}")


총 110개 항목 제안됨
                  original                           suggested
0           Ankle Pumping1          frontal__ankle_pumping.mp4
1           Ankle Pumping2          lateral__ankle_pumping.mp4
2           Ankle Pumping3         diagonal__ankle_pumping.mp4
3             Biceps Curl1            frontal__biceps_curl.mp4
4             Biceps Curl2            lateral__biceps_curl.mp4
5             Biceps Curl3           diagonal__biceps_curl.mp4
6            Biceps Curl_1            frontal__biceps_curl.mp4
7            Biceps Curl_2            lateral__biceps_curl.mp4
8            Biceps Curl_3           diagonal__biceps_curl.mp4
9   Biceps Curl_Bilateral1   frontal__biceps_curlbilateral.mp4
10  Biceps Curl_Bilateral2   lateral__biceps_curlbilateral.mp4
11  Biceps Curl_Bilateral3  diagonal__biceps_curlbilateral.mp4
12         Bridge_Dynamic1         frontal__bridge_dynamic.mp4
13         Bridge_Dynamic2         lateral__bridge_dynamic.mp4
14         Bridge_Dynamic3        diagona

In [7]:
import pandas as pd
from pathlib import Path

# 1️⃣ 매핑 CSV 경로
csv_path = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/0_RAW_DATA/AI_dataset/mapping_suggestions.csv")
df = pd.read_csv(csv_path)

# 2️⃣ 제외 항목: N으로 시작하는 모든 항목 (띄어쓰기 또는 _ 포함) + titles_cleaned
exclude_mask = (
    df["original"].str.match(r"^N\d+[\s_]", case=False) | 
    df["original"].str.contains("titles_cleaned", case=False)
)
df = df[~exclude_mask]

# 3️⃣ 행동(Action) 이름 추출
df["action_name"] = (
    df["suggested"]
    .str.split("__").str[-1]
    .str.replace(".mp4", "", regex=False)
)

# 4️⃣ 고유 행동 개수 계산
unique_actions = sorted(df["action_name"].unique())

# 5️⃣ 결과 출력
print(f"✅ (병실·치료실·titles_cleaned 완전 제외) 고유 행동(Action) 개수: {len(unique_actions)}개")
print("=" * 50)
for act in unique_actions:
    print(act)


✅ (병실·치료실·titles_cleaned 완전 제외) 고유 행동(Action) 개수: 29개
ankle_pumping
biceps_curl
biceps_curlbilateral
bridge_dynamic
bridge_static
chest_press
clamshell_sidelying
clamshell_supine
hip_abduction
hip_extension
hip_flexion_extension
hip_flexion_extension_bobath_table
hip_flexion_extension_chair
hip_flexion_extension_wheelchair
hip_knee_flexion
knee_extension_chair
knee_extension_wheelchair
knee_flexion
leg_cycle_exercise
lumbar_rotation
one_leg_bridge_dynamic
one_leg_bridge_static
overhead_triceps_extension
pec_dec_fly
q_setting
shoulder_abduction
shoulder_flexion
slr
slr_bobath_table


In [ ]:
# ============================================================
# ✅ mapping_suggestions_cleaned.csv 기준으로 실제 파일 이름 일괄 변경
#     - base_dir 하위 모든 하위폴더 포함
#     - 매핑되지 않은 파일은 print로 표시
# ============================================================

from pathlib import Path
import pandas as pd

# 1️⃣ 기본 경로
csv_dir = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/mapping_suggestions.csv")

# 2️⃣ 매핑 CSV 불러오기
csv_path = base_dir / "mapping_suggestions.csv"
df = pd.read_csv(csv_path)

# 3️⃣ 매핑 딕셔너리 생성 {original_name: suggested_name}
mapping_dict = dict(zip(df["original"], df["suggested"]))

# 4️⃣ 모든 .mp4 / .mov 파일 탐색 (대소문자 무시)
video_exts = [".mp4", ".mov"]
all_files = [
    f for f in base_dir.rglob("*")
    if f.is_file()
    and f.suffix.lower() in video_exts
    and "@eaDir" not in f.parts
    and not f.name.endswith("@SynoEAStream")
]

print(f"📂 Found {len(all_files)} video files to check")

# 5️⃣ 매핑 및 이름 변경
unmapped = []
for f in all_files:
    stem = f.stem  # 확장자 제외 원본 이름
    if stem in mapping_dict:
        new_name = mapping_dict[stem]
        new_path = f.with_name(new_name)
        if new_path.exists():
            print(f"⚠️ 이미 존재: {new_name} → 건너뜀")
            continue
        try:
            f.rename(new_path)
            print(f"✅ {f.name} → {new_name}")
        except Exception as e:
            print(f"❌ {f.name} 변경 실패: {e}")
    else:
        unmapped.append(f.name)

# 6️⃣ 매핑되지 않은 파일 출력
print("\n⚠️ 매핑되지 않은 파일 목록:")
if unmapped:
    for name in unmapped:
        print(" -", name)
    print(f"\n총 {len(unmapped)}개 파일 매핑되지 않음")
else:
    print("✅ 모든 파일이 매핑 완료됨!")


📂 Found 944 video files to check
✅ Biceps Curl1.MP4 → frontal__biceps_curl.mp4
✅ Biceps Curl2.MOV → lateral__biceps_curl.mp4
✅ Biceps Curl3.MP4 → diagonal__biceps_curl.mp4
✅ Hip Extension1.MP4 → frontal__hip_extension.mp4
✅ Hip Extension2.MOV → lateral__hip_extension.mp4
✅ Hip Extension3.MP4 → diagonal__hip_extension.mp4
✅ Hip Flexion Extension1.MP4 → frontal__hip_flexion_extension.mp4
✅ Hip Flexion Extension2.MOV → lateral__hip_flexion_extension.mp4
✅ Hip Flexion Extension3.MP4 → diagonal__hip_flexion_extension.mp4
✅ Hip Flexion Extension_bobathtable1.MOV → frontal__hip_flexion_extension_bobath_table.mp4
✅ Hip Flexion Extension_bobathtable2.MP4 → lateral__hip_flexion_extension_bobath_table.mp4
✅ Hip Flexion Extension_bobathtable3.MP4 → diagonal__hip_flexion_extension_bobath_table.mp4
✅ Hip Flexion Extension_chair1.MP4 → frontal__hip_flexion_extension_chair.mp4
✅ Hip Flexion Extension_chair2.MOV → lateral__hip_flexion_extension_chair.mp4
✅ Hip Flexion Extension_chair3.MP4 → diagonal__h

In [10]:
# ============================================================
# ✅ 1_FRAME/AI_dataset 하위 2단계 폴더 이름 일괄 매핑
#     - mapping_suggestions.csv 기준
#     - *_frames 접미사는 제거
#     - 하위 파일은 무시 (폴더 이름만 변경)
# ============================================================

from pathlib import Path
import pandas as pd

# 1️⃣ 경로 설정
base_dir = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/1_FRAME/AI_dataset")
csv_dir  = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/mapping_suggestions.csv")

# 2️⃣ 매핑 CSV 불러오기
df = pd.read_csv(csv_dir)
mapping_dict = dict(zip(df["original"], df["suggested"]))

# 3️⃣ AI_dataset 하위의 2단계 폴더만 탐색 (예: N01/N01_Treatment/*)
treatment_dirs = [
    subdir for person_dir in base_dir.iterdir()
    if person_dir.is_dir()
    for subdir in person_dir.iterdir()
    if subdir.is_dir() and "@eaDir" not in subdir.parts
]

print(f"📂 Found {len(treatment_dirs)} treatment folders (2-depth)")

# 4️⃣ *_frames 폴더만 필터링
target_dirs = [
    d for t_dir in treatment_dirs for d in t_dir.iterdir()
    if d.is_dir() and d.name.endswith("_frames") and "@eaDir" not in d.parts
]

print(f"📁 Found {len(target_dirs)} target '_frames' folders")

# 5️⃣ 매핑 및 이름 변경
unmapped = []

for d in target_dirs:
    stem = d.name.replace("_frames", "")  # "_frames" 제거
    if stem in mapping_dict:
        new_name = mapping_dict[stem].replace(".mp4", "")  # 확장자 제거
        new_path = d.with_name(new_name)
        if new_path.exists():
            print(f"⚠️ 이미 존재: {new_name} → 건너뜀")
            continue
        try:
            d.rename(new_path)
            print(f"✅ {d.name} → {new_name}")
        except Exception as e:
            print(f"❌ {d.name} 변경 실패: {e}")
    else:
        unmapped.append(d.name)

# 6️⃣ 매핑되지 않은 폴더 출력
print("\n⚠️ 매핑되지 않은 폴더 목록:")
if unmapped:
    for name in unmapped:
        print(" -", name)
    print(f"\n총 {len(unmapped)}개 폴더 매핑되지 않음")
else:
    print("✅ 모든 폴더가 매핑 완료됨!")


📂 Found 20 treatment folders (2-depth)
📁 Found 942 target '_frames' folders
✅ Biceps Curl1_frames → frontal__biceps_curl
✅ Biceps Curl2_frames → lateral__biceps_curl
✅ Biceps Curl3_frames → diagonal__biceps_curl
✅ Hip Extension1_frames → frontal__hip_extension
✅ Hip Extension2_frames → lateral__hip_extension
✅ Hip Extension3_frames → diagonal__hip_extension
✅ Hip Flexion Extension1_frames → frontal__hip_flexion_extension
✅ Hip Flexion Extension2_frames → lateral__hip_flexion_extension
✅ Hip Flexion Extension3_frames → diagonal__hip_flexion_extension
✅ Hip Flexion Extension_Wheelchair1_frames → frontal__hip_flexion_extension_wheelchair
✅ Hip Flexion Extension_Wheelchair2_frames → lateral__hip_flexion_extension_wheelchair
✅ Hip Flexion Extension_bobathtable1_frames → frontal__hip_flexion_extension_bobath_table
✅ Hip Flexion Extension_bobathtable2_frames → lateral__hip_flexion_extension_bobath_table
✅ Hip Flexion Extension_bobathtable3_frames → diagonal__hip_flexion_extension_bobath_table

In [11]:
# ============================================================
# ✅ 2_KEYPOINTS/AI_dataset 하위 폴더 이름 일괄 매핑
#     - mapping_suggestions.csv 기준
#     - *_JSON 접미사는 제거
#     - Synology @eaDir 폴더 무시
#     - 하위 파일은 건드리지 않음
# ============================================================

from pathlib import Path
import pandas as pd

# 1️⃣ 경로 설정
base_dir = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/2_KEYPOINTS/AI_dataset")
csv_dir  = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/mapping_suggestions.csv")

# 2️⃣ 매핑 CSV 불러오기
df = pd.read_csv(csv_dir)
mapping_dict = dict(zip(df["original"], df["suggested"]))

# 3️⃣ AI_dataset 하위의 2단계 폴더만 탐색 (예: N01/N01_Treatment/*)
treatment_dirs = [
    subdir for person_dir in base_dir.iterdir()
    if person_dir.is_dir()
    for subdir in person_dir.iterdir()
    if subdir.is_dir() and "@eaDir" not in subdir.parts
]

print(f"📁 Found {len(treatment_dirs)} treatment folders (2-depth)")

# 4️⃣ *_JSON 폴더만 필터링
target_dirs = [
    d for t_dir in treatment_dirs for d in t_dir.iterdir()
    if d.is_dir() and d.name.endswith("_JSON") and "@eaDir" not in d.parts
]

print(f"🎯 Found {len(target_dirs)} target '_JSON' folders")

# 5️⃣ 매핑 및 이름 변경
unmapped = []

for d in target_dirs:
    stem = d.name.replace("_JSON", "")  # "_JSON" 제거
    if stem in mapping_dict:
        new_name = mapping_dict[stem].replace(".mp4", "")  # 확장자 제거
        new_path = d.with_name(new_name)
        if new_path.exists():
            print(f"⚠️ 이미 존재: {new_name} → 건너뜀")
            continue
        try:
            d.rename(new_path)
            print(f"✅ {d.name} → {new_name}")
        except Exception as e:
            print(f"❌ {d.name} 변경 실패: {e}")
    else:
        unmapped.append(d.name)

# 6️⃣ 매핑되지 않은 폴더 출력
print("\n⚠️ 매핑되지 않은 폴더 목록:")
if unmapped:
    for name in unmapped:
        print(" -", name)
    print(f"\n총 {len(unmapped)}개 폴더 매핑되지 않음")
else:
    print("✅ 모든 폴더가 매핑 완료됨!")

print("\n🏁 2_KEYPOINTS JSON 폴더 처리 완료")


📁 Found 20 treatment folders (2-depth)
🎯 Found 942 target '_JSON' folders
✅ Biceps Curl1_JSON → frontal__biceps_curl
✅ Biceps Curl2_JSON → lateral__biceps_curl
✅ Biceps Curl3_JSON → diagonal__biceps_curl
✅ Hip Extension1_JSON → frontal__hip_extension
✅ Hip Extension2_JSON → lateral__hip_extension
✅ Hip Extension3_JSON → diagonal__hip_extension
✅ Hip Flexion Extension1_JSON → frontal__hip_flexion_extension
✅ Hip Flexion Extension2_JSON → lateral__hip_flexion_extension
✅ Hip Flexion Extension3_JSON → diagonal__hip_flexion_extension
✅ Hip Flexion Extension_Wheelchair1_JSON → frontal__hip_flexion_extension_wheelchair
✅ Hip Flexion Extension_Wheelchair2_JSON → lateral__hip_flexion_extension_wheelchair
✅ Hip Flexion Extension_bobathtable1_JSON → frontal__hip_flexion_extension_bobath_table
✅ Hip Flexion Extension_bobathtable2_JSON → lateral__hip_flexion_extension_bobath_table
✅ Hip Flexion Extension_bobathtable3_JSON → diagonal__hip_flexion_extension_bobath_table
✅ Hip Flexion Extension_chair

In [12]:
# ============================================================
# ✅ 3_MP4/AI_dataset 하위 mp4 파일 이름 일괄 매핑
#     - mapping_suggestions.csv 기준
#     - 확장자 .mp4 대소문자 구분 없음
#     - Synology @eaDir 폴더 무시
# ============================================================

from pathlib import Path
import pandas as pd

# 1️⃣ 경로 설정
base_dir = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/3_MP4/AI_dataset")
csv_dir  = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/mapping_suggestions.csv")

# 2️⃣ 매핑 CSV 불러오기
df = pd.read_csv(csv_dir)
mapping_dict = dict(zip(df["original"], df["suggested"]))

# 3️⃣ AI_dataset 하위의 2단계 폴더만 탐색 (예: N01/N01_Treatment)
treatment_dirs = [
    subdir for person_dir in base_dir.iterdir()
    if person_dir.is_dir()
    for subdir in person_dir.iterdir()
    if subdir.is_dir() and "@eaDir" not in subdir.parts
]

print(f"📁 Found {len(treatment_dirs)} treatment folders (2-depth)")

# 4️⃣ mp4 파일만 필터링 (대소문자 무시)
target_files = [
    f for t_dir in treatment_dirs for f in t_dir.iterdir()
    if f.is_file() and f.suffix.lower() == ".mp4" and "@eaDir" not in f.parts
]

print(f"🎯 Found {len(target_files)} target '.mp4' files")

# 5️⃣ 매핑 및 이름 변경
unmapped = []

for f in target_files:
    stem = f.stem
    if stem in mapping_dict:
        new_name = mapping_dict[stem]
        new_path = f.with_name(new_name)
        if new_path.exists():
            print(f"⚠️ 이미 존재: {new_name} → 건너뜀")
            continue
        try:
            f.rename(new_path)
            print(f"✅ {f.name} → {new_name}")
        except Exception as e:
            print(f"❌ {f.name} 변경 실패: {e}")
    else:
        unmapped.append(f.name)

# 6️⃣ 매핑되지 않은 파일 출력
print("\n⚠️ 매핑되지 않은 파일 목록:")
if unmapped:
    for name in unmapped:
        print(" -", name)
    print(f"\n총 {len(unmapped)}개 파일 매핑되지 않음")
else:
    print("✅ 모든 파일이 매핑 완료됨!")

print("\n🏁 3_MP4 mp4 파일 처리 완료")


📁 Found 20 treatment folders (2-depth)
🎯 Found 942 target '.mp4' files
✅ Biceps Curl1.mp4 → frontal__biceps_curl.mp4
✅ Biceps Curl2.mp4 → lateral__biceps_curl.mp4
✅ Biceps Curl3.mp4 → diagonal__biceps_curl.mp4
✅ Hip Extension1.mp4 → frontal__hip_extension.mp4
✅ Hip Extension2.mp4 → lateral__hip_extension.mp4
✅ Hip Extension3.mp4 → diagonal__hip_extension.mp4
✅ Hip Flexion Extension1.mp4 → frontal__hip_flexion_extension.mp4
✅ Hip Flexion Extension2.mp4 → lateral__hip_flexion_extension.mp4
✅ Hip Flexion Extension3.mp4 → diagonal__hip_flexion_extension.mp4
✅ Hip Flexion Extension_Wheelchair1.mp4 → frontal__hip_flexion_extension_wheelchair.mp4
✅ Hip Flexion Extension_Wheelchair2.mp4 → lateral__hip_flexion_extension_wheelchair.mp4
✅ Hip Flexion Extension_bobathtable1.mp4 → frontal__hip_flexion_extension_bobath_table.mp4
✅ Hip Flexion Extension_bobathtable2.mp4 → lateral__hip_flexion_extension_bobath_table.mp4
✅ Hip Flexion Extension_bobathtable3.mp4 → diagonal__hip_flexion_extension_bobath_

# metadata update

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
build_metadata_from_raw.py

📌 /0_RAW_DATA 하위의 비디오를 기준으로 metadata_new2.csv 생성
- video_path, frame_path, keypoints_path, mp4_path 자동 추출
- 각 폴더 존재 여부, 파일 개수(n_frames, n_json) 자동 계산
- 모든 *_done은 True로 저장 (초기 세팅)
- is_train, is_val은 False로 초기화
"""

from pathlib import Path
import pandas as pd

# ---------------- 경로 설정 ----------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove")
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "0_RAW_DATA"
FRAME_DIR = DATA_DIR / "1_FRAME"
KEYPOINT_DIR = DATA_DIR / "2_KEYPOINTS"
MP4_DIR = DATA_DIR / "3_MP4"
CSV_OUT = DATA_DIR / "metadata_new2.csv"

# ---------------- 비디오 파일 탐색 ----------------
video_exts = [".mp4", ".MP4", ".mov", ".MOV"]
video_files = [p for p in RAW_DIR.rglob("*") if p.suffix in video_exts]

records = []
print(f"[INFO] 총 {len(video_files)}개 비디오 파일 탐색됨.")

for video_path in video_files:
    rel_video_path = video_path.relative_to(DATA_DIR)

    # 비디오 기준 상대 이름
    rel_noext = rel_video_path.with_suffix("")  # 확장자 제거
    frame_dir = FRAME_DIR / rel_noext.relative_to("0_RAW_DATA")
    key_dir = KEYPOINT_DIR / rel_noext.relative_to("0_RAW_DATA")
    mp4_path = MP4_DIR / rel_noext.relative_to("0_RAW_DATA")
    mp4_path = mp4_path.with_suffix(".mp4")

    # 파일 개수 계산
    n_frames = len(list(frame_dir.glob("*.jpg"))) if frame_dir.exists() else 0
    n_json = len(list(key_dir.glob("*.json"))) if key_dir.exists() else 0

    # CSV 한 행 구성
    record = {
        "video_path": str(rel_video_path),
        "frame_path": str(frame_dir.relative_to(DATA_DIR)) if frame_dir.exists() else "",
        "keypoints_path": str(key_dir.relative_to(DATA_DIR)) if key_dir.exists() else "",
        "mp4_path": str(mp4_path.relative_to(DATA_DIR)) if mp4_path.exists() else "",
        "n_frames": n_frames,
        "n_json": n_json,
        "frames_done": frame_dir.exists(),
        "sapiens_done": key_dir.exists(),
        "reextract_done": key_dir.exists(),
        "overlay_done": mp4_path.exists(),
        "is_train": False,
        "is_val": False
    }
    records.append(record)

# ---------------- CSV 저장 ----------------
df = pd.DataFrame(records)
df.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")

print(f"[INFO] metadata_new2.csv 생성 완료 → {CSV_OUT}")
print(f"[INFO] 총 {len(df)}개 행 기록됨.")


[INFO] 총 1158개 비디오 파일 탐색됨.
[INFO] metadata_new2.csv 생성 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/metadata_new2.csv
[INFO] 총 1158개 행 기록됨.


In [3]:
import pandas as pd
from pathlib import Path

# 경로 설정
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data")
old_path = BASE_DIR / "metadata_new.csv"
new_path = BASE_DIR / "metadata_new2.csv"
out_path = BASE_DIR / "metadata_new3.csv"   # 결과 저장 파일명

# CSV 불러오기
old_df = pd.read_csv(old_path)
new_df = pd.read_csv(new_path)

# is_train이 True인 video_path 목록 추출
train_videos = set(old_df.loc[old_df["is_train"] == True, "video_path"])

# 일치하는 video_path에 대해 is_train 갱신
new_df["is_train"] = new_df["video_path"].apply(lambda x: True if x in train_videos else new_df.loc[new_df["video_path"] == x, "is_train"].values[0])

# 결과 저장
new_df.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"[INFO] is_train 업데이트 완료 → {out_path}")
print(f"[INFO] True로 설정된 샘플 수: {(new_df['is_train'] == True).sum()}개")


[INFO] is_train 업데이트 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/metadata_new3.csv
[INFO] True로 설정된 샘플 수: 53개


In [4]:
import pandas as pd
from pathlib import Path

# 경로 설정
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data")
csv_path = BASE_DIR / "metadata_new3.csv"
out_path = BASE_DIR / "metadata_new3_sorted.csv"

# CSV 불러오기
df = pd.read_csv(csv_path)

# video_path 기준 정렬
df_sorted = df.sort_values(by="video_path", ascending=True).reset_index(drop=True)

# 저장
df_sorted.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"[INFO] metadata_new3_sorted.csv 생성 완료 → {out_path}")
print(f"[INFO] 총 {len(df_sorted)}개 행 정렬됨.")


[INFO] metadata_new3_sorted.csv 생성 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/metadata_new3_sorted.csv
[INFO] 총 1158개 행 정렬됨.
